In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('africa_merged_dataset.csv')
solar = pd.read_csv('../../energy_strength/solar/solar_countries.csv')
solar = solar[solar['iso_a3'].isin(list(df['iso3_code']))]
wind = pd.read_csv('../../energy_strength/wind/wind_countries.csv')
wind = wind[wind['iso_a3'].isin(list(df['iso3_code']))]


In [10]:
df.sample(5)
# get rows count of df
# print(df.shape[0])

,lat,long,population,wind_speed,solar,iso2_code,name,iso3_code,corruption_perception_index,conflict_intensity
440527,-12.500000,14.166667,556.80480,3.735886,253.720901,AO,Angola,AGO,29,240
413592,-7.500000,14.541667,47.18614,3.466038,219.564026,AO,Angola,AGO,29,240
153127,11.958333,3.125000,1473.72200,4.954491,262.387665,BJ,Benin,BEN,42,268
25665,30.375000,23.875000,44.96635,6.099619,202.255997,LY,Libya,LBY,17,2245
99288,21.125000,25.958333,6.67907,6.380187,277.415070,LY,Libya,LBY,17,2245


In [11]:
solar.sample(5)

,Unnamed: 0,lat,lon,ALLSKY_SFC_SW_DWN,name,iso_a3
24331,24331,-22.5,31.5,220.242188,Mozambique,MOZ
39770,39770,20.5,-9.5,265.804688,Mauritania,MRT
35842,35842,9.5,22.5,246.160156,Central African Rep.,CAF
27201,27201,-14.5,21.5,237.746094,Angola,AGO
31171,31171,-3.5,31.5,228.113281,Tanzania,TZA


In [ ]:
wind.sample(5)

,Unnamed: 0,lat,lon,WS50M,name,iso_a3
108628,108628,4.0,32.500,3.230469,S. Sudan,SSD
136861,136829,28.5,18.125,5.855469,Libya,LBY
85573,85573,-16.0,23.125,4.750000,Zambia,ZMB
113748,113748,8.5,-7.500,3.945312,Côte d'Ivoire,CIV
81560,81560,-19.5,35.000,4.917969,Mozambique,MOZ


In [20]:
# df.drop(['wind_speed', 'solar'], axis=1, inplace=True)
import numpy as np
from scipy.spatial import distance
def generateDatasetNodeList(datasetDF, targetVar):
    nodes = []
    nodeWVar = []
    for index, row in datasetDF.iterrows():
        nodes.append((row['lat'], row['lon']))
        nodeWVar.append((row['lat'], row['lon'], row[targetVar]))
    return np.asarray(nodes), nodeWVar

solarNodes, solarNodesWVar = generateDatasetNodeList(solar, 'ALLSKY_SFC_SW_DWN')
windNodes, windNodesWVar = generateDatasetNodeList(wind, 'WS50M')

def closest_node(node, nodes):
    distVals = distance.cdist([node], nodes)
    minIdx = distVals.argmin()
    # closest_index = distance.cdist([node], nodes).argmin()
    return minIdx, distVals[0][minIdx]

solarThreshold = distance.euclidean((0, 0), (1, 1)) * 1.25
windThreshold = distance.euclidean((0, 0), (0.5, 0.625)) * 1.25

# import cProfile

# pr = cProfile.Profile()
# iterate through df and for each row, replace wind_speed and solar with 0
# pr.enable()
finalList = []
for index, row in df.iterrows():
    if index % 10000 == 0 and index != 0:
        print(index)
        # pr.disable()
        # import pstats, io
        # from pstats import SortKey
        # s = io.StringIO()
        # sortby = SortKey.CUMULATIVE
        # ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        # ps.print_stats()
        # print(s.getvalue())
        # break
    closestIdx, dist = closest_node((row['lat'], row['long']), solarNodes)
    if dist < solarThreshold:
        solarVal = solarNodesWVar[closestIdx][2]
    else:
        solarVal = np.NaN

    closestIdx, dist = closest_node((row['lat'], row['long']), windNodes)
    if dist < windThreshold:
        windVal = windNodesWVar[closestIdx][2]
    else:
        windVal = np.NaN

    finalList.append([row['lat'], row['long'], solarVal, windVal])
    # df.loc[index, 'solar'] = 0
    # df.loc[index, 'wind_speed'] = 0


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000


In [22]:
solarWindDF = pd.DataFrame(finalList, columns=['lat', 'long', 'solar', 'wind_speed'])

# drop solar and wind_speed columns from df
df.drop(['wind_speed', 'solar'], axis=1, inplace=True)

# merge df and solarWindDF on lat and long
df = df.merge(solarWindDF, on=['lat', 'long'], how='left')

# write df to a csv
df.to_csv('africa_merged_dataset_nasa_power.csv', index=False)

,lat,long,population,iso2_code,name,iso3_code,corruption_perception_index,conflict_intensity,solar,wind_speed
0,37.583333,8.916667,12.795480,TN,Tunisia,TUN,44,1345,205.167969,6.062500
1,37.583333,8.958333,0.047371,TN,Tunisia,TUN,44,1345,205.167969,6.062500
2,37.541667,8.833333,1.305495,TN,Tunisia,TUN,44,1345,205.167969,6.062500
3,37.541667,8.875000,139.428800,TN,Tunisia,TUN,44,1345,205.167969,6.062500
4,37.541667,8.916667,387.129000,TN,Tunisia,TUN,44,1345,205.167969,6.062500
...,...,...,...,...,...,...,...,...,...,...
625517,-30.708333,27.875000,23.237460,LS,Lesotho,LSO,38,80,242.769531,5.679688
625518,-30.708333,27.916667,23.237450,LS,Lesotho,LSO,38,80,242.769531,5.679688
625519,-30.708333,27.958333,20.565640,LS,Lesotho,LSO,38,80,242.769531,5.679688
625520,-30.708333,28.000000,17.867160,LS,Lesotho,LSO,38,80,242.769531,5.679688
